In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import os
from matplotlib import pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
import matplotlib.image as mpimg
from keras import Sequential
from keras.layers import Conv2D,Dense,MaxPooling2D,BatchNormalization,Flatten,Dropout
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [2]:
base_dir, _ = os.path.splitext("../../../../Downloads/CellData/chest_xray")

In [3]:
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir,'test')

train_normal = os.path.join(train_dir, 'NORMAL')
print ('Total training normal images:', len(os.listdir(train_normal)))
train_pneu = os.path.join(train_dir, 'PNEUMONIA')
print ('Total training pneu images:', len(os.listdir(train_pneu)))


test_normal = os.path.join(test_dir, 'NORMAL')
print ('Total test normal images:', len(os.listdir(test_normal)))
test_pneu = os.path.join(test_dir, 'PNEUMONIA')
print ('Total test pneu images:', len(os.listdir(test_pneu)))

Total training normal images: 1349
Total training pneu images: 3884
Total test normal images: 235
Total test pneu images: 390


In [4]:
image_size = 224 # All images will be resized to 160x160
batch_size = 32

# Rescale all images by 1./255 and apply image augmentation
train_datagen = keras.preprocessing.image.ImageDataGenerator(validation_split = .2,rescale=1./255)

test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
                train_dir,  # Source directory for the training images
                subset='training',
                target_size=(image_size, image_size),
                batch_size=batch_size,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='binary')

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = train_datagen.flow_from_directory(
                train_dir, # Source directory for the validation images
                subset='validation', # select validation subset
                target_size=(image_size, image_size),
                batch_size=batch_size,
                class_mode='binary',
                shuffle = False)

# Flow validation images in batches of 20 using test_datagen generator
test_generator = test_datagen.flow_from_directory(
                test_dir, # Source directory for the validation images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                class_mode='binary',
                shuffle=False)

Found 4187 images belonging to 2 classes.
Found 1045 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [5]:
model = Sequential()

In [6]:
model.add(Conv2D(32, kernel_size = (3, 3), activation='relu', input_shape=(image_size, image_size, 3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())

model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())

model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())


model.add(Conv2D(96, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(.3))


model.add(Conv2D(32, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.3))
model.add(Dense(1, activation = 'sigmoid'))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



In [7]:
model.compile(optimizer = SGD(learning_rate = .001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [9]:
epochs = 30
steps_per_epoch = train_generator.n // batch_size
validation_steps = validation_generator.n // batch_size
# lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=2, verbose=2, mode='max')
es = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
cb_list = [es]
history = model.fit(train_generator,
                              steps_per_epoch = steps_per_epoch,
                              epochs=epochs,
                              validation_data=validation_generator,
                              validation_steps=validation_steps,
                              callbacks = cb_list )


Epoch 1/30
130/130 [==============================] - 148s 1s/step - loss: 0.4236 - accuracy: 0.8130 - val_loss: 0.1984 - val_accuracy: 0.7373
Epoch 2/30
130/130 [==============================] - 157s 1s/step - loss: 0.2821 - accuracy: 0.8823 - val_loss: 0.0746 - val_accuracy: 0.7345
Epoch 3/30
130/130 [==============================] - 169s 1s/step - loss: 0.2275 - accuracy: 0.9134 - val_loss: 0.0136 - val_accuracy: 0.7345
Epoch 4/30
130/130 [==============================] - 166s 1s/step - loss: 0.2034 - accuracy: 0.9203 - val_loss: 0.0258 - val_accuracy: 0.7345
Epoch 5/30
130/130 [==============================] - 164s 1s/step - loss: 0.1765 - accuracy: 0.9312 - val_loss: 0.0231 - val_accuracy: 0.8006
Epoch 6/30
130/130 [==============================] - 160s 1s/step - loss: 0.1645 - accuracy: 0.9372 - val_loss: 0.0104 - val_accuracy: 0.8470
Epoch 7/30
130/130 [==============================] - 161s 1s/step - loss: 0.1527 - accuracy: 0.9408 - val_loss: 0.2886 - val_accuracy: 0.932

In [11]:
# model.save('bigdatamodel.h5')

In [12]:
predictions = []
for x in model.predict(test_generator):
    for z in x:
        predictions.append(np.round(z))
        
metrics.confusion_matrix(test_generator.classes,predictions)

array([[136,  98],
       [  1, 389]])

In [20]:
y_pred_new =(model.predict(test_generator)>0.66).astype(int)
metrics.confusion_matrix(test_generator.classes,y_pred_new)

array([[159,  75],
       [  5, 385]])